In [1]:
import psycopg2
#import fileinput
from lib.database import Database

#p is a predicate
def get_rule(p):  
    # test yagoB.predicate = '<wasCreatedOnDate>' for subjects which are not people; else '<wasBornOnDate>'
    timestamp_query = """ SELECT yagoA.id, yagoA.subject, yagoA.predicate, yagoA.object, yagoB.predicate, yagoB.object
                            FROM yagofacts yagoA
                                 JOIN yagofacts yagoB ON yagoA.object = yagoB.subject
                            where yagoA.predicate = %s
                            AND yagoB.predicate = %s
                            ORDER BY yagoA.subject
                            """
    
    if p in ['actedIn', 'directed', 'wroteMusicFor', 'created', 'participatedIn', 'hasChild']:
        return timestamp_query
    elif p in ['worksAt', 'isPoliticianOf', 'graduatedFrom', 'isMarriedTo', 'playsFor', 'isLeaderOf', 'isKnownFor']:
         return 'interval_time'
    else:
        return 0

#rule can be a sql query or a function's name
def apply_rule(rule, predicate, idx):
    if rule == 'interval_time':
        return interval_time(predicate, idx)
    
    range_predicate = ['<wasBornOnDate>', '<wasCreatedOnDate>']
    conn = None
    try:
        conn = Database.connect_dbs()
        cur = conn.cursor()
        if idx == 0:
            cur.execute(rule, (predicate, range_predicate[0], )) #e.g., hasChild
        else:
            cur.execute(rule, (predicate, range_predicate[1], )) #e.g., actedIn, directed
            
        data = cur.fetchall()      
        
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    
    return data

/home/joana/virtualenvs/yagoapp/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


## Interval Time

In [2]:
"""
b1 d1 b2 d2 with intersection (OMITED)
b1 b2 d1 d2 without
b1 b2 d2 d1 contained
"""
def getdates(sborn, sdied, oborn, odied):
    sborndate = strtodate(sborn)
    sdieddate = strtodate(sdied)
    oborndate = strtodate(oborn)
    odieddate = strtodate(odied)
    if oborndate != None and sborndate != None and sborndate <= oborndate:
        strfirst = [sborn, sdied]
        strsecond = [oborn, odied]
        first = [sborndate, sdieddate]
        second = [oborndate, odieddate]
    else:
        strfirst = [oborn, odied]
        strsecond = [sborn, sdied]
        first = [oborndate, odieddate]
        second = [sborndate, sdieddate]
    
    if first[1] != None and second[0] != None and first[1] < second[0]: # No intersection
        return None, None
    
    if first[0] == None and second[0] == None: # unknown born or created dated of both entities
        return None, None
    
    # NPYN -> NY
    # NPNY -> NY
    # NPYQ -> NY
    if first[0] == None and second[0] != None and first[1] != None:
        if second[1] != None and second[1] < first[1]:
            return None, None
        else:
            return None, strfirst[1]
        
    # XNNQ -> NQ
    # XNQN -> NQ
    # XNQY -> NQ
    if first[0] != None and second[0] == None and second[1] != None:
        if first[1] != None and first[1] < second[1]:
            return None, None
        else:
            return None, strsecond[1]
        
    if first[0] != None and second[0] != None:
        if first[1] != None and second[1] != None:
            if first[1] < second[1]:# XPYQ -> PY
                return strsecond[0], strfirst[1]
            else: # XPQY -> PQ
                return strsecond[0], strsecond[1]
        
        # XPYN -> PY
        # XPNY -> PY
        if first[1] != None:
            return strsecond[0], strfirst[1]
        
        # XPNQ -> PQ
        # XPQN -> PQ
        if second[1] != None:
            return strsecond[0], strsecond[1]

        # XPNN -> PN
        # XPNN -> PN
        return strsecond[0], None
        
    return None, None
    
    # xyz
    """
    if first[1] != None and second[1] != None:
        if second[1] < first[1]:
            return second[0], second[1]
        else:
            return second[0], first[1]
    elif second[1] == None:
        return second[0], first[1]
    else:
        return second[0], second[1]"""
    

In [3]:
def interval_time(predicate, idx=0):    
    date_predicate_born = ['<wasBornOnDate>', '<wasCreatedOnDate>']
    date_predicate_died = ['<diedOnDate>', '<wasDestroyedOnDate>']
    
    query = """SELECT yagoA.id, yagoA.subject, yagoA.predicate, yagoA.object
                FROM yagofacts yagoA
                where yagoA.predicate = %s
                ORDER BY yagoA.subject"""

    qsborn = "select * from yagofacts where subject = %s and predicate = '<wasBornOnDate>'"
    qsdied = "select * from yagofacts where subject = %s and predicate = '<diedOnDate>'"
    qoborn = "select * from yagofacts where subject = %s and predicate = %s"
    qodied = "select * from yagofacts where subject = %s and predicate = %s"
    conn = None
    # content = []
    content = {}
    try:
        conn = Database.connect_dbs()
        cur = conn.cursor()  
        
        cur.execute(query, (predicate, ))
        data = cur.fetchall()
        for row in data:
            
            cursb = conn.cursor()
            cursb.execute(qsborn, (row[1], ))
            dateinfo = cursb.fetchall()
            if len(dateinfo) <= 0:
                sborn = None
            else:
            #    print("sborn", dateinfo[0][3])
                sborn = parse_date(dateinfo[0][3])
            cursb.close()
            
            cursb = conn.cursor()
            cursb.execute(qsdied, (row[1], ))
            dateinfo = cursb.fetchall()
            if len(dateinfo) <= 0:
                sdied = None
            else:
             #   print("sdied", dateinfo[0][3])
                sdied = parse_date(dateinfo[0][3])
            cursb.close()
            
            cursb = conn.cursor()
            cursb.execute(qoborn, (row[3], date_predicate_born[idx], ))
            dateinfo = cursb.fetchall()
            if len(dateinfo) <= 0:
                oborn = None
            else:
              #  print("oborn", dateinfo[0][3])
                oborn = parse_date(dateinfo[0][3])
            cursb.close()
            
            cursb = conn.cursor()
            cursb.execute(qodied, (row[3], date_predicate_died[idx], ))
            dateinfo = cursb.fetchall()
            if len(dateinfo) <= 0:
                odied = None
            else:
              #  print("odied", dateinfo[0][3])
                odied = parse_date(dateinfo[0][3])
            cursb.close()            
                       
            if sborn == '' or sdied == '' or oborn == '' or odied == '':
                continue
            
            if (sdied != None and sborn != None) and sdied < sborn:
                continue
            
            if  (odied != None and oborn != None) and odied < oborn:
                continue
                
           # print("-> <%s> <%s> <%s> <%s> "%(sborn if sborn != None else "N", sdied if sdied != None else "N", oborn if oborn!= None else "N", odied if odied!= None else "N"))
            validAfter, validBefore = getdates(sborn, sdied, oborn, odied)
                
            info = {
                'id': row[0],#p[0],
                'name': row[1], 
                'predicate': predicate,
                'company': row[3],
                'after': "after",
                'validAfter': validAfter.lstrip('0') if validAfter!=None else None,
                'before': "before",
                'validBefore': validBefore.lstrip('0') if validBefore!=None else None
            }
                # Organize records to compose <s,p,o> after d1 before d2 without duplicates
            key = '%s%s'%(row[1], row[3])
            if not key in content:
                content[key] = info
            else:
                if content[key]['validAfter'] is None:
                    content[key]['validAfter'] = info.get('validAfter')

                if content[key]['validBefore'] is None:
                    content[key]['validBefore'] = info.get('validBefore')   
        
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        traceback.print_exc()
    finally:
        if conn is not None:
            conn.close()
    return content

In [4]:
import json
import csv 
import os
import fileinput
from utils import *

if __name__ == '__main__':    
    
    predicates = [['isMarriedTo', 'hasChild'],
                  ['actedIn', 'directed', 'wroteMusicFor', 'created', 'participatedIn', 'worksAt', 
                   'isPoliticianOf', 'graduatedFrom', 'playsFor', 'isLeaderOf', 'isKnownFor']]
    file_name = ""
    count = 0
    for i in range(2):
        for p in predicates[i]:
            rule = get_rule(p)
            file_name = p
            p = '<'+p+'>'
            if rule:      
                results = apply_rule(rule, p, i)
                if isinstance(results, dict):
                    with open("outputs/"+file_name+".csv", "w+") as f:
                        w = csv.writer(f, delimiter = '\t')
                        for key in results:
                            tmp = []
                            for item in results[key].items():
                                tmp.append(converttostr(item[1]))
                            w.writerow(tmp)
                        """for info in results:
                            tmp = []
                            for item in info.items():
                                tmp.append(converttostr(item[1]))
                            w.writerow(tmp)""" 
                    f.close()
                else:
                    file = open("outputs/"+file_name+".txt", "w+")
                    for result in results:
                        file.write(str(result)+"\n")
                    file.close()
        
        with fileinput.FileInput("outputs/"+file_name+".txt", inplace=True, backup='.bak') as file:
            if p in ['<actedIn>', '<directed>', '<wroteMusicFor>', '<created>', '<participatedIn>']:
                for line in file:
                    print(line.replace("wasCreatedOnDate", "inDateTime"), end='')
            elif p in ['<hasChild>']:
                for line in file:
                    print(line.replace("wasBornOnDate", "inDateTime"), end='')
                
    
    #Evaluation- in progress